In [1]:
from ipymlwidgets import ImageAnnotated, Box, Text, Button
from ipymlwidgets.demo import demo_image
import numpy as np

image = ImageAnnotated(image=demo_image())
box = Box([image], layout={"width": "50%", "border": "1px solid black", "border-radius": "10px"})
display(box)







Box(children=[MultiCanvas((32, 32), layers=3)], layout={'width': '50%', 'border': '1px solid black', 'border-r…

In [ ]:
from ipymlwidgets import ImageAnnotated, Box, Text, Button
from ipymlwidgets.demo import demo_image
from pathlib import Path
import numpy as np
from torchsorcery.dataset import DatasetManager, DatasetItem
import ipywidgets as W
import traceback

info = W.HTML("<p>Test</p>")
info.value = "<p>Ok</p>"
display(info)


def debug_observer(func):
    """Decorator that catches and prints all exceptions"""
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            info.value = f"<p>{traceback.format_exc()}</p>"
            raise e
    return wrapper

PATH = Path("D:/.dataset/certops/r5040-sample-1/")
DatasetManager.disable_caching()
dataset = DatasetManager.load_dataset(PATH, use_cache=False, include_meta=True)['test']
dataset = dataset.map(lambda item: dict(**item['data'], file=item['_file']), remove_columns=dataset.column_names) # remove column names is required...

INDEX = 0

def save_example(example, base_dir : Path):
    # just going to save the meta data - the boxes!
    path = base_dir / example['file']
    item = DatasetItem.load(path)
    boxes = ia.boxes.copy().astype(np.long)
    item.data['boxes'] = [{"box" : boxes[i].tolist()} for i in range(len(boxes))]

    # get data from the latest
    display(str(item))
    item.save(path, force=True)
    
@debug_observer
def on_prev_click(_):
    global INDEX
    save_example(get_example(INDEX), PATH)
    INDEX -= 1
    example = get_example(INDEX)
    ia.set_image(example['image'])
    ia.set_boxes(example['boxes'])

def on_key_press(event):
    event = event['new']
    global INDEX
    display(str(event))
    if event['key'] == 'ArrowRight':
        on_next_click(None)
    elif event['key'] == 'ArrowLeft':
        on_next_click(None)
        
@debug_observer
def get_example(index):
    """Get an example from the dataset."""
    example = dataset[index]
    image = np.array(example['image'])
    anno = example.get("boxes", [])
    anno = anno if anno else []
    boxes = []
    texts = []
    for item in anno:
        boxes.append(item['box'])
        texts.append(item.get('text', item.get('label', '')))
    boxes = np.array(boxes)
    boxes = boxes.reshape(boxes.shape[0], 4) # handle empty boxes
    return dict(image=image, boxes=boxes, texts=texts, file=example['file'])

@debug_observer
def on_next_click(_):
    info.value = "<p>1</p>"

    global INDEX
    info.value = "<p>2</p>"

    save_example(get_example(INDEX), PATH)
    info.value = "<p>3</p>"

    INDEX += 1 
    info.value = "<p>3.5</p>"

    example = get_example(INDEX)
    ia.set_image(example['image'])
    ia.set_boxes(example['boxes'])
    
example = get_example(INDEX) # initial example
ia = ImageAnnotated(example['image'], example['boxes'])
ia.observe(on_key_press, "key_press")

btn_next, btn_prev = Button("Next"), Button("Prev")
btn_next.on_click(on_next_click)
btn_prev.on_click(on_prev_click)
btn_box = Box([btn_prev, btn_next], layout={"display": "flex", "flex-direction": "row", "padding": "10px", "width": "100%"})

el = Box([ia, btn_box], layout={"width": "100%"})

def on_selection(change):
    pass # TODO

ia.observe(on_selection, "selection")

display(el)

In [ ]:
from ipymlwidgets import ImageAnnotated, Box, Text, Button
from ipymlwidgets.demo import demo_image
from pathlib import Path
import numpy as np
from torchsorcery.dataset import DatasetManager, DatasetItem
import ipywidgets as W

PATH = Path("D:/.dataset/certops/r5040-sample-1/")
DatasetManager.disable_caching()
dataset = DatasetManager.load_dataset(PATH, use_cache=False, include_meta=True)['test']
dataset = dataset.map(lambda item: dict(**item['data'], file=item['_file']), remove_columns=dataset.column_names) # remove column names is required...


In [ ]:

from torchsorcery.dataset import DatasetManager
from pathlib import Path
PATH = Path("D:/.dataset/certops/r5040-sample-1/")

import datasets
datasets.disable_caching()
dataset = DatasetManager.load_dataset(PATH, include_meta=True, use_cache=False)['test']
print(dataset.features)

for i in dataset:
    print(i)